<img src="https://raw.githubusercontent.com/UXARRAY/uxarray/main/docs/_static/images/logos/uxarray_logo_h_dark.svg"
     width="30%"
     alt="HEALPix logo"
     align="right"
/>

# UXarray for Advanced HEALPix Analysis & Visualization

### In this section, you'll learn:

* Using the `uxarray` package to perform advanced analysis operators over HEALPix data such as non-conservative zonal means, etc.

### Related Documentation

* [UXarray homepage](https://uxarray.readthedocs.io/en/latest/index.html)
* [Working with HEALPix data - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/healpix.html)
* [UXarray overview - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/02-intro-to-uxarray/overview.html)
* [Data visualization with UXarray - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/03-plotting-with-uxarray/data-viz.html)
* [Cross-sections - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/cross-sections.html)
* [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [UXarray](https://uxarray.readthedocs.io/en/latest/index.html) | Necessary  | |
| [HEALPix overview](00-healpix) | Necessary  | |

**Time to learn**: 30 minutes

-----

In [ ]:
import cartopy.crs as ccrs
import holoviews as hv
import intake
import uxarray as ux

## Open data catalog

:::{tip} We assume, you have already gone over the previous section, [UXarray for Basic HEALPix Statistics & Visualization](02-uxarray). If not and if you need to learn about data catalogs in general and the data we will use throughout this notebook, we recommend to check that section first.:::

Let us open the online catalog from the [WCRP's Digital Earths Global Hackathon 2025](https://digital-earths-global-hackathon.github.io/) using `intake` and read the output of the `ICON` run `ngc4008`, which is stored in the HEALPix format:

In [ ]:
# Final data catalog location (once hackathon website (https://digital-earths-global-hackathon.github.io/) updated)
# cat_url='https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml'
# Interim data catalog location
cat_url = "https://raw.githubusercontent.com/digital-earths-global-hackathon/catalog/refs/heads/ncar/online/main.yaml"
cat = intake.open_catalog(cat_url)
model_run = cat.icon_ngc4008

We can look into a fine resolution dataset at **zoom level = 10** in it as `Xarray.Dataset`:

In [ ]:
ds = model_run(zoom=9, time="P1D").to_dask()
ds

### Create UXarray Datasets from HEALPix

We can use `from_healpix` as follows to open a HEALPix grid from `xarray.Dataset`:

In [ ]:
uxds = ux.UxDataset.from_healpix(ds)
uxds

### Data variable of interest

Then let us pick a variable from the dataset, which will give us an `uxarray.UxDataArray`:

In [ ]:
uxda = uxds["tas"]
uxda

### Global mean and plot

Computing the global air temperature mean (at the first timestep) and also having a quick plot of it would be a good idea to have as references to compare the upcoming analyses & visualizations to them:

In [ ]:
%%time
print(
    "Global air temperature average on ", uxda.time[0].values, ": ", uxda.isel(time=0).mean().values, " K"
)

In [ ]:
%%time

projection = ccrs.Robinson()

uxda.isel(time=0).plot(
    projection=projection,
    cmap="inferno",
    features=["borders", "coastline"],
    title="Global temperature",
    width=700,
)

## Rasterized point plots

When working with a higher-resolution dataset at a global scale, it's not always practical to render each cell as a polygon. Instead, we can rasterize the center of each pixel.

In [ ]:
projection = ccrs.Robinson()

# Controls the size of each pixel (smaller value leads to larger pixels)
pixel_ratio = 0.5

uxda.isel(time=0).plot.points(
    projection=projection,
    rasterize=True,
    dynamic=False,
    width=1000,
    height=500,
    pixel_ratio=pixel_ratio,
    cmap="inferno",
    title=f"Global Air Temperature, pixel_ratio={pixel_ratio}",
)

If we decrease the size of each pixel (by setting the pixel ratio to a higher value), we can start to see missing values, which is due to a lower density of points near the poles, leading to some pixels not containing any of our original points.

Because of this, it's useful to try a few `pixel_ratio` values and see which one works best for your given resolution.

In [ ]:
projection = ccrs.Robinson()

# Controls the size of each pixel (smaller value leads to larger pixels)
pixel_ratio = 2.0

uxda.isel(time=0).plot.points(
    projection=projection,
    rasterize=True,
    dynamic=False,
    width=1000,
    height=500,
    pixel_ratio=pixel_ratio,
    cmap="inferno",
    title=f"Global Air Temperature with bad pixel size selection, pixel_ratio={pixel_ratio}",
)

## Cross-sections

We can look at constant latitude/longitude cross-sections of an `uxarray.UxDataArray`:

In [ ]:
boulder_lat = 40.0190


# With fine resolutions like zoom level of 9, it is visually hard to observe the cross-sections,
# so we will use a zoom level of 4 for a better visualization
uxda_coarse = ux.UxDataset.from_healpix(model_run(zoom=4, time="P1D").to_dask())["tas"]
uxda_coarse.uxgrid.face_node_connectivity

uxda_lat = uxda_coarse.cross_section.constant_latitude(boulder_lat)
uxda_lat

In [ ]:
import geoviews.feature as gf

uxda_lat.isel(time=0).plot(
    rasterize=False,
    projection=projection,
    global_extent=True,
    cmap="inferno",
    clim=(220, 310),
    features=["coastline"],
    title=f"Global temperature cross-section at {boulder_lat} degrees latitude",
    width=700,
) * gf.grid(projection=projection)

Let's also look at the mean of the cross-section:

In [ ]:
print(
    f"Mean at {boulder_lat} degrees lat (Boulder, CO, USA): {uxda_lat.mean().values} K"
)

### Latitude interval

In [ ]:
uxda_lat_interval = uxda_coarse.cross_section.constant_latitude_interval(
    [boulder_lat - 15, boulder_lat + 15]
)

In [ ]:
uxda_lat_interval.isel(time=0).plot(
    rasterize=False,
    projection=projection,
    global_extent=True,
    cmap="inferno",
    clim=(220, 310),
    features=["coastline"],
    title=f"Global temperature cross-section at the latitude interval [{boulder_lat-5},{boulder_lat+5}] degrees",
    width=700,
) * gf.grid(projection=projection)

In [ ]:
print(
    f"Mean at the latitude interval of [{boulder_lat-5},{boulder_lat+5}] degrees (-/+15 degrees Boulder, CO, USA): {uxda_lat_interval.mean().values} K"
)

## Non-conservative zonal mean

Calculating the zonal mean is easy by providing the latitude range between -90 and 90 degrees with a step size in degrees:

In [ ]:
zonal_mean_tas = uxda.isel(time=0).zonal_mean(lat=(-90, 90, 5))

In [ ]:
(
    uxda.isel(time=0).plot(
        cmap="inferno",
        # periodic_elements="split",
        height=300,
        width=600,
        colorbar=False,
        ylim=(-90, 90),
    )
    + zonal_mean_tas.plot.line(
        x="tas_zonal_mean",
        y="latitudes",
        height=300,
        width=180,
        ylabel="",
        ylim=(-90, 90),
        xlim=(220, 310),
        # xticks=[220, 250, 280, 310],
        yticks=[-90, -45, 0, 45, 90],
        grid=True,
    )
).opts(title="Temperature and its Zonal means at every 5 degrees latitude")

## Remapping

Now, we will be looking into a remapping case. The data set we are using in this section has the **zoom=10** available but not for all the variables, e.g. `tas` has all NaN values at that zoom level. Let us try to remap our data here that is sampled at **zoom=9** to a `Grid` at **zoom=10**.

Let's start with creating the destination `uxarray.Grid`:

In [ ]:
%%time
uxgrid_zoom10 = ux.Grid.from_healpix(zoom=10, pixels_only=False)

In [ ]:
%%time
uxda_remapped = uxda.isel(time=0).remap.inverse_distance_weighted(
    uxgrid_zoom10, k=3, remap_to="face centers"
)

In [ ]:
%%time

uxda_remapped.plot(
    projection=projection,
    cmap="inferno",
    features=["borders", "coastline"],
    title="Global temperature - remapped to zoom=10",
    width=700,
)